In [1]:
import pandas as pd
import numpy as np
import datetime

hoje = datetime.date.today()

In [2]:
tres_dias = [
    hoje - datetime.timedelta(days=1),
    hoje - datetime.timedelta(days=2),
    hoje - datetime.timedelta(days=3)
]

In [6]:
ocorrencias = pd.read_excel('tbl_base_GDO_2020.xls', sheet_name='tbl_crimes')
ocorrencias.loc[:,'Ano'] = ocorrencias.loc[:,'Data Fato'].dt.year
ocorrencias.loc[:,'Dia'] = ocorrencias.loc[:,'Data Fato'].dt.day
df

WARNING *** file size (38313540) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


0         1
1         1
2         1
3         1
4         1
         ..
52770     6
52771    10
52772    20
52773    13
52774    15
Name: Data Fato, Length: 52775, dtype: int64

In [7]:
roubos_furtos_tres_dias = ocorrencias[
    ( ocorrencias['Código Subclasse Nat Principal'].isin(['C01157', 'C01155']) )
    & (     
            (
                (
                    (ocorrencias['Data Fato'].dt.year.isin([tres_dias[0].year, tres_dias[0].year -1]) )
                    & (ocorrencias['Data Fato'].dt.month == tres_dias[0].month)
                    & (ocorrencias['Data Fato'].dt.day == tres_dias[0].day)            
                )
                |
                (
                    (ocorrencias['Data Fato'].dt.year.isin([tres_dias[0].year, tres_dias[1].year -1]) )
                    & (ocorrencias['Data Fato'].dt.month == tres_dias[1].month)
                    & (ocorrencias['Data Fato'].dt.day == tres_dias[1].day)            
                )
                |
                (
                    (ocorrencias['Data Fato'].dt.year.isin([tres_dias[0].year, tres_dias[2].year -1]) )
                    & (ocorrencias['Data Fato'].dt.month == tres_dias[2].month)
                    & (ocorrencias['Data Fato'].dt.day == tres_dias[2].day)            
                )
                
            )
        
    )
                
]

In [9]:
comparativo_dias = pd.pivot_table(
    data=roubos_furtos_tres_dias,
    index='Município',
    columns=['Dia', 'Ano'],
    values='Qtde Ocorrências',
    aggfunc='sum'
)

comparativo_dias

Dia                       15         16        17
Ano                     2019 2020  2019 2020 2019
Município                                        
ABAETE                   4.0  NaN   1.0  NaN  1.0
ARAUJOS                  NaN  NaN   NaN  NaN  1.0
ARCOS                    2.0  NaN   NaN  NaN  5.0
BAMBUI                   NaN  2.0   1.0  NaN  NaN
BOM DESPACHO             1.0  NaN   2.0  1.0  1.0
CARMO DO CAJURU          NaN  NaN   3.0  NaN  2.0
CLAUDIO                  NaN  NaN   2.0  NaN  1.0
DIVINOPOLIS             11.0  3.0  12.0  1.0  4.0
DORES DO INDAIA          1.0  NaN   NaN  NaN  NaN
FORMIGA                  2.0  4.0   3.0  1.0  NaN
IGUATAMA                 NaN  NaN   2.0  NaN  NaN
ITAPECERICA              NaN  NaN   NaN  NaN  1.0
ITATIAIUCU               NaN  1.0   1.0  NaN  2.0
ITAUNA                   6.0  1.0   9.0  NaN  4.0
LAGOA DA PRATA           1.0  2.0   NaN  NaN  1.0
LUZ                      1.0  NaN   1.0  NaN  4.0
MARAVILHAS               NaN  NaN   1.0  NaN  NaN
MARTINHO CAMPOS          NaN  NaN   1.0  NaN  2.0
NOVA SERRANA             7.0  NaN  13.0  2.0  4.0
PAPAGAIOS                NaN  NaN   1.0  NaN  1.0
PARA DE MINAS            7.0  NaN   2.0  NaN  3.0
PERDIGAO                 2.0  NaN   NaN  NaN  NaN
PIMENTA                  1.0  NaN   NaN  NaN  NaN
PITANGUI                 NaN  NaN   2.0  NaN  NaN
POMPEU                   NaN  1.0   1.0  NaN  4.0
SANTO ANTONIO DO MONTE   NaN  1.0   2.0  NaN  2.0
SAO GONCALO DO PARA      2.0  1.0   NaN  NaN  1.0
SAO JOSE DA VARGINHA     NaN  NaN   2.0  NaN  NaN
SAO SEBASTIAO DO OESTE   NaN  NaN   NaN  NaN  1.0

In [115]:
comparativo_anos = pd.pivot_table(
    data=ocorrencias[
        ocorrencias['Ano'].isin((2019, 2020))
    ],
    index='Município',
    columns='Ano',
    values='Qtde Ocorrências',
    aggfunc='sum'
)

comparativo_anos_multi_index = pd.MultiIndex.from_product([
    ['Ano'], comparativo_anos.columns
])

comparativo_anos.columns = comparativo_anos_multi_index

In [117]:
%%time
pd.concat([
    comparativo_dias,
    comparativo_anos,
],axis=1)\
.fillna(0)\
.astype('int')\
.sort_index()

CPU times: user 7.46 ms, sys: 0 ns, total: 7.46 ms
Wall time: 6.85 ms


Dia                      14        15        16        Ano      
Ano                    2019 2020 2019 2020 2019 2020  2019  2020
ABAETE                    0    0    4    0    1    0   352   253
ARAUJOS                   0    0    0    0    0    0    72    46
ARCOS                     3    2    2    0    0    0   454   259
BAMBUI                    0    0    0    2    1    0   297   172
BIQUINHAS                 0    0    0    0    0    0    20     6
BOM DESPACHO              2    0    1    0    2    1   906   447
CAMACHO                   0    0    0    0    0    0    12     1
CARMO DO CAJURU           0    0    0    0    3    0   429   207
CEDRO DO ABAETE           0    0    0    0    0    0    16     5
CLAUDIO                   3    0    0    0    2    0   422   254
CONCEICAO DO PARA         0    0    0    0    0    0    98    44
CORREGO DANTA             0    0    0    0    0    0    39    26
CORREGO FUNDO             0    0    0    0    0    0    35    12
DIVINOPOLIS               8   10   11    3   12    1  5218  2470
DORES DO INDAIA           0    0    1    0    0    0   196    88
ESTRELA DO INDAIA         0    0    0    0    0    0    20     3
FORMIGA                   2    3    2    4    3    1   814   450
IGARATINGA                1    0    0    0    0    0   115    53
IGUATAMA                  1    0    0    0    2    0    75    19
ITAPECERICA               1    0    0    0    0    0   228   143
ITATIAIUCU                0    0    0    1    1    0   265   171
ITAUNA                    4    1    6    1    9    0  1917   877
JAPARAIBA                 0    0    0    0    0    0    46    20
LAGOA DA PRATA            3    0    1    2    0    0   800   452
LEANDRO FERREIRA          0    0    0    0    0    0    37    26
LUZ                       0    0    1    0    1    0   311   145
MARAVILHAS                0    0    0    0    1    0    55    25
MARTINHO CAMPOS           0    1    0    0    1    0   225   110
MEDEIROS                  0    0    0    0    0    0    17     7
MOEMA                     0    0    0    0    0    0   114    58
MORADA NOVA DE MINAS      0    0    0    0    0    0   138    75
NOVA SERRANA              2    1    7    0   13    2  2297  1184
ONCA DO PITANGUI          0    0    0    0    0    0    31    15
PAINEIRAS                 0    0    0    0    0    0    29    11
PAINS                     0    0    0    0    0    0    61    28
PAPAGAIOS                 2    0    0    0    1    0   203    71
PARA DE MINAS             3    1    7    0    2    0  1509   582
PEDRA DO INDAIA           0    0    0    0    0    0    48    23
PEQUI                     0    1    0    0    0    0    43    23
PERDIGAO                  0    0    2    0    0    0   165   117
PIMENTA                   0    0    1    0    0    0   129    55
PITANGUI                  0    1    0    0    2    0   353   160
POMPEU                    1    0    0    1    1    0   504   248
QUARTEL GERAL             1    0    0    0    0    0    43    12
SANTO ANTONIO DO MONTE    0    0    0    1    2    0   346   203
SAO GONCALO DO PARA       0    0    2    1    0    0   177   137
SAO JOSE DA VARGINHA      0    0    0    0    2    0    56    23
SAO SEBASTIAO DO OESTE    1    0    0    0    0    0   102    42
SERRA DA SAUDADE          0    0    0    0    0    0     3     2
TAPIRAI                   0    0    0    0    0    0    11     3

In [120]:
%time
comparativo_dias.merge(comparativo_anos, on='Município', how='outer')\
.fillna(0).astype('int')

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15 µs


Dia                      14        15        16        Ano      
Ano                    2019 2020 2019 2020 2019 2020  2019  2020
Município                                                       
ABAETE                    0    0    4    0    1    0   352   253
ARCOS                     3    2    2    0    0    0   454   259
BAMBUI                    0    0    0    2    1    0   297   172
BOM DESPACHO              2    0    1    0    2    1   906   447
CARMO DO CAJURU           0    0    0    0    3    0   429   207
CLAUDIO                   3    0    0    0    2    0   422   254
DIVINOPOLIS               8   10   11    3   12    1  5218  2470
DORES DO INDAIA           0    0    1    0    0    0   196    88
FORMIGA                   2    3    2    4    3    1   814   450
IGARATINGA                1    0    0    0    0    0   115    53
IGUATAMA                  1    0    0    0    2    0    75    19
ITAPECERICA               1    0    0    0    0    0   228   143
ITATIAIUCU                0    0    0    1    1    0   265   171
ITAUNA                    4    1    6    1    9    0  1917   877
LAGOA DA PRATA            3    0    1    2    0    0   800   452
LUZ                       0    0    1    0    1    0   311   145
MARAVILHAS                0    0    0    0    1    0    55    25
MARTINHO CAMPOS           0    1    0    0    1    0   225   110
NOVA SERRANA              2    1    7    0   13    2  2297  1184
PAPAGAIOS                 2    0    0    0    1    0   203    71
PARA DE MINAS             3    1    7    0    2    0  1509   582
PEQUI                     0    1    0    0    0    0    43    23
PERDIGAO                  0    0    2    0    0    0   165   117
PIMENTA                   0    0    1    0    0    0   129    55
PITANGUI                  0    1    0    0    2    0   353   160
POMPEU                    1    0    0    1    1    0   504   248
QUARTEL GERAL             1    0    0    0    0    0    43    12
SANTO ANTONIO DO MONTE    0    0    0    1    2    0   346   203
SAO GONCALO DO PARA       0    0    2    1    0    0   177   137
SAO JOSE DA VARGINHA      0    0    0    0    2    0    56    23
SAO SEBASTIAO DO OESTE    1    0    0    0    0    0   102    42
ARAUJOS                   0    0    0    0    0    0    72    46
BIQUINHAS                 0    0    0    0    0    0    20     6
CAMACHO                   0    0    0    0    0    0    12     1
CEDRO DO ABAETE           0    0    0    0    0    0    16     5
CONCEICAO DO PARA         0    0    0    0    0    0    98    44
CORREGO DANTA             0    0    0    0    0    0    39    26
CORREGO FUNDO             0    0    0    0    0    0    35    12
ESTRELA DO INDAIA         0    0    0    0    0    0    20     3
JAPARAIBA                 0    0    0    0    0    0    46    20
LEANDRO FERREIRA          0    0    0    0    0    0    37    26
MEDEIROS                  0    0    0    0    0    0    17     7
MOEMA                     0    0    0    0    0    0   114    58
MORADA NOVA DE MINAS      0    0    0    0    0    0   138    75
ONCA DO PITANGUI          0    0    0    0    0    0    31    15
PAINEIRAS                 0    0    0    0    0    0    29    11
PAINS                     0    0    0    0    0    0    61    28
PEDRA DO INDAIA           0    0    0    0    0    0    48    23
SERRA DA SAUDADE          0    0    0    0    0    0     3     2
TAPIRAI                   0    0    0    0    0    0    11     3